In [2]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from openai import OpenAI
client = OpenAI()

In [ ]:


def get_embedding(text, client):
    response = client.embeddings.create(
        input = text,
        model = "text-embedding-3-small")
    return response.data[0].embedding

#usage

emb = get_embedding("hello", client)

In [7]:
def get_summary(text, client):
    
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant. You will be given a text and you will summarize it. Provide only the summary of the text without any additional information."},
        {
            "role": "user",
            "content": f"TEXT: {text}"
        }
    ]
    )

    return completion.choices[0].message.content

text = "La sesta battaglia dell'Isonzo, conosciuta anche come battaglia o presa di Gorizia, fu combattuta dal 4 al 17 agosto 1916 sul fronte italiano della prima guerra mondiale, e vide contrapposti il Regio Esercito italiano al comando del generale Luigi Cadorna e l'Imperiale e regio esercito austro-ungarico al comando del generale Franz Conrad von Hötzendorf.Inizialmente prevista per metà giugno, Cadorna dovette rinviare l'inizio della Sesta battaglia dell'Isonzo a causa dell'inaspettata offensiva austro-ungarica in Tirolo del maggio 1916, la Südtiroloffensive. Sferrata a maggio da Conrad, l'offensiva mirava ad invadere la pianura veneta sfondando le linee italiane dal saliente Trentino, con lo scopo di prendere alle spalle e tagliare la ritirata al grosso dell'esercito italiano stanziato quasi interamente lungo il medio e basso Isonzo, giungendo in questo modo alla fine della"

get_summary(text, client)

"La sesta battaglia dell'Isonzo, nota anche come la presa di Gorizia, si svolse dal 4 al 17 agosto 1916 sul fronte italiano della Prima Guerra Mondiale, tra il Regio Esercito italiano guidato dal generale Luigi Cadorna e l'esercito austro-ungarico sotto il generale Franz Conrad von Hötzendorf. Originariamente prevista per giugno, la battaglia fu posticipata a causa dell'offensiva austro-ungarica in Tirolo di maggio 1916, che mirava a invadere la pianura veneta e tagliare le retrovie delle forze italiane."

In [ ]:
### generazione chunk contestualizzati (migliora la ricerca):
# 1. generare un riassunto dell'intero documento
# 2. strutturare un prompt con il riassunto dell'intero documento, il chunk da contestualizzare e gli N chunk precedenti (e successivi?)
# 3. dare istruzioni al modello di generare un riassunto del chunk utilizzando il contesto fornito
# 4. fare embedding del riassunto del chunk (sia bm25 che gpt)
# 5. utilizzare embedding per rag

#### rispondere a domande complesse
# 1. prendere la domanda e scomporla in step
# 2. utilizzare il react agent per generare una risposta per ogni step
# 3. combinare le risposte in una risposta complessiva

### funzionamento react agent
# 1. il react agent ha a sua disposizione 2 tool
# 1.1 il tool che data una domanda fa il retrival dei top K chunks dall'intero corpus di documenti
# 1.2 il tool che dato un documento e un id di chunk, fornisce una porzione di documento intorno al chunk e il riassunto del documento
# 2. ci si aspetta che il funzionamento atteso del react agent sia il seguente:
# 2.1 il react agent riceve una domanda
# 2.2 il react agent utilizza il tool 1 per recuperare i top K chunks
# 2.3 il react agent utilizza il tool 2 per recuperare il contesto del chunk
# 2.4 se il contesto basta per rispondere alla domanda il react agent risponde, altrimenti chiede al tool 1 di recuperare altri chunks oppure richiede il contesto di altri chunk